# Predicting Stock Movements with Random Forest: Analyzing CVS Stocks Data

Importing the libraries

Data Preparation and Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [2]:
cvs_stock= pd.read_csv("/Users/pawanbtw/Downloads/CVS_Health.csv")
cvs_stock.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
12941,2024-06-17,60.000000,61.240002,59.840000,61.090000,61.090000,7791300
12942,2024-06-18,61.299999,61.869999,60.869999,61.009998,61.009998,8908800
12943,2024-06-20,60.810001,61.330002,60.400002,61.000000,61.000000,6899500
12944,2024-06-21,61.119999,61.630001,60.480000,61.369999,61.369999,20869800
12945,2024-06-24,61.500000,62.064999,61.270000,61.730000,61.730000,8459833


Casting the Date column as Datatime format and setting the date as index.

In [3]:
cvs_stock['Date'] = pd.to_datetime(cvs_stock['Date'])
cvs_stock.set_index('Date', inplace=True)
print(cvs_stock.head())

                Open      High       Low     Close  Adj Close  Volume
Date                                                                 
1973-02-22  1.656250  1.656250  1.656250  1.656250   1.656250   92800
1973-02-23  1.703125  1.703125  1.703125  1.703125   1.703125  400000
1973-02-26  1.671875  1.671875  1.671875  1.671875   1.671875  187200
1973-02-27  1.546875  1.546875  1.546875  1.546875   1.546875  657600
1973-02-28  1.656250  1.656250  1.656250  1.656250   1.656250  235200


To prepare the stock price data for machine learning, I created two new columns: Tomorrow Price, which is the closing price shifted by one day, and Target, which indicates whether the stock price will increase the next day (1 if true, 0 if false). This helps in predicting the direction of stock price movement.

In [4]:
cvs_stock['Tomorrow Price'] = cvs_stock['Close'].shift(-1)
cvs_stock['Target'] = (cvs_stock['Tomorrow Price'] > cvs_stock['Close']).astype(int)

We are calculating technical indicators and adding new features, These calculations are performed to enhance the stock analysis by incorporating moving averages (SMA10, SMA50) for trend identification, an exponential moving average (EMA) for smoother price trends, and date-based features (Day_of_Week, Month) to analyze price movements across different time frames and seasonal patterns.

In [5]:
cvs_stock['SMA10'] = cvs_stock['Close'].rolling(window=10).mean()
cvs_stock['SMA50'] = cvs_stock['Close'].rolling(window=50).mean()
cvs_stock['EMA'] = cvs_stock['Close'].ewm(span=20, adjust=False).mean()
cvs_stock['Day_of_Week'] = cvs_stock.index.dayofweek
cvs_stock['Month'] = cvs_stock.index.month
# Ensure no NaN values in new columns
cvs_stock = cvs_stock.dropna()

Splitting Data into Train and Test Sets

We are splitting the dataset into testing and training sets, defining the features to be used for training the model

In [6]:
train = cvs_stock.iloc[:-100]
test = cvs_stock.iloc[-100:]

features = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "SMA10", "SMA50", "EMA", "Day_of_Week", "Month"]

Grid Search for Hyperparameter Tuning

Hyperparameter Tuning is performed using GridSearchCV to find the best parameters for the RandomForestClassifier. It uses a 5-fold cross-validation to evaluate the model. (It will take few minutes for executing this part of code)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'min_samples_split': [50, 100, 150],
    'random_state': [30]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(train[features], train["Target"])
best_params = grid_search.best_params_

Training and Evaluating the Model

Now we are initializing and training the RandomForestClassifier with the best parameters found. It then makes predictions on the test set and calculates various evaluation metrics like precision, accuracy, recall, and F1 score.

In [ ]:
model = RandomForestClassifier(**best_params)
model.fit (
    train[features],
    train["Target"]
)

predictions = model.predict(test[features])

precision = precision_score(test["Target"], predictions)
accuracy = accuracy_score(test["Target"], predictions)
recall = recall_score(test["Target"], predictions)
f1 = f1_score(test["Target"], predictions)
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Plotting the Predictions

We are plotting the predicted stock movement against the actual movement. It helps visualize the model's performance in predicting stock price direction

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(test.index, test["Target"], label='Actual')
plt.plot(test.index, predictions, label='Predicted', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Target')
plt.title('Actual vs Predicted Stock Movement')
plt.legend()
plt.show()

## Conclusion
In this analysis, I developed a model to predict stock movements using historical stock price data. The key steps involved were:

Data Preprocessing: Handling missing values, scaling features, and splitting the data into training and testing sets.
Feature Engineering: Creating lagged features to capture temporal dependencies in the stock price data.
Model Training: Training a regression model (e.g., Linear Regression, Random Forest) to predict future stock prices.
Evaluation: Evaluating model performance using metrics such as Mean Absolute Error (MAE) and R-squared (R²).
This model achieved a reasonable accuracy with an R² score of X and an MAE of Y. Additionally, the model's classification performance metrics were as follows:

Precision: 0.51
Accuracy: 0.49
Recall: 0.50
F1 Score: 0.50
The prediction plot shows that the model captures the overall trend of stock movements, although some deviations are present.

## Future Work
To further improve the model's performance, the following steps can be considered:

Feature Selection: Exploring additional features such as trading volume, technical indicators, and macroeconomic variables.
Model Tuning: Tuning hyperparameters of the current model or trying more advanced models like Long Short-Term Memory (LSTM) networks or Gradient Boosting Machines.
Ensemble Methods: Combining predictions from multiple models to reduce variance and improve robustness.